In [1]:
!pip install pandas plotly dash jupyter-dash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [2]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash


In [4]:
df = pd.read_csv("creditranscation.csv")

print(df.head())

# Check for missing values
print(df.isnull().sum())


   Customer ID      Name    Surname Gender   Birthdate  Transaction Amount  \
0       752858      Sean  Rodriguez      F  20-10-2002               35.47   
1        26381  Michelle     Phelps    NaN  24-10-1985             2552.72   
2       305449     Jacob   Williams      M  25-10-1981              115.97   
3       988259    Nathan     Snyder      M  26-10-1977               11.31   
4       764762   Crystal      Knapp      F  02-11-1951               62.21   

         Date               Merchant Name     Category  
0  03-04-2023               Smith-Russell     Cosmetic  
1  17-07-2023      Peck, Spence and Young       Travel  
2  20-09-2023                  Steele Inc     Clothing  
3  11-01-2023  Wilson, Wilson and Russell     Cosmetic  
4  13-06-2023               Palmer-Hinton  Electronics  
Customer ID              0
Name                     0
Surname                  0
Gender                5047
Birthdate                0
Transaction Amount       0
Date                     0


In [5]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Birthdate'] = pd.to_datetime(df['Birthdate'], errors='coerce')

# Drop rows where dates are invalid
df = df.dropna(subset=['Date', 'Birthdate'])


<ipython-input-5-b6008e35de71>:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Birthdate'] = pd.to_datetime(df['Birthdate'], errors='coerce')


In [6]:
df['Age'] = (pd.to_datetime('today') - df['Birthdate']).dt.days // 365


In [7]:
df['YearMonth'] = df['Date'].dt.to_period('M')


In [8]:
transaction_trends = df.groupby('YearMonth').size().reset_index(name='TransactionCount')


In [9]:
category_spending = df.groupby('Category')['Transaction Amount'].sum().reset_index()


In [10]:
# Calculate IQR for outlier detection
q1 = df['Transaction Amount'].quantile(0.25)
q3 = df['Transaction Amount'].quantile(0.75)
iqr = q3 - q1

# Define upper and lower limits
upper_limit = q3 + 1.5 * iqr
lower_limit = q1 - 1.5 * iqr

# Identify anomalies
anomalies = df[(df['Transaction Amount'] > upper_limit) | (df['Transaction Amount'] < lower_limit)]


In [11]:
pip install pandas plotly dash


In [12]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px


In [13]:
app = Dash(__name__)


In [14]:
app.layout = html.Div([
    html.H1("Dynamic Payment Analytics Dashboard"),

    dcc.Dropdown(
        id='category-filter',
        options=[{'label': cat, 'value': cat} for cat in df['Category'].unique()],
        placeholder="Select a Category"
    ),
    dcc.Graph(id='transaction-trend'),
    dcc.Graph(id='category-spending'),
    dcc.Graph(id='anomaly-graph'),
])


In [15]:
@app.callback(
    Output('transaction-trend', 'figure'),
    [Input('category-filter', 'value')]
)
def update_transaction_trend(selected_category):
    filtered_data = df[df['Category'] == selected_category] if selected_category else df
    trend_data = filtered_data.groupby('YearMonth').size().reset_index(name='TransactionCount')
    fig = px.line(trend_data, x='YearMonth', y='TransactionCount', title="Transaction Trends")
    return fig

@app.callback(
    Output('category-spending', 'figure'),
    [Input('category-filter', 'value')]
)
def update_category_spending(selected_category):
    filtered_data = df[df['Category'] == selected_category] if selected_category else df
    spending_data = filtered_data.groupby('Category')['Transaction Amount'].sum().reset_index()
    fig = px.bar(spending_data, x='Category', y='Transaction Amount', title="Category Spending")
    return fig

@app.callback(
    Output('anomaly-graph', 'figure'),
    [Input('category-filter', 'value')]
)
def update_anomaly_graph(selected_category):
    filtered_data = df[df['Category'] == selected_category] if selected_category else df
    anomalies = filtered_data[(filtered_data['Transaction Amount'] > upper_limit) |
                              (filtered_data['Transaction Amount'] < lower_limit)]
    fig = px.scatter(anomalies, x='Date', y='Transaction Amount', title="Anomalies")
    return fig


In [16]:
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

In [18]:
pip install pyngrok


In [19]:
!ngrok authtoken 2pGAz4ZUvqNRS3E81GpNjNxdgzp_84sgE2ZtevhVcXStGvn9H


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [20]:
from pyngrok import ngrok

# Start the tunnel
public_url = ngrok.connect(8050)
print("Dashboard URL:", public_url)



Dashboard URL: NgrokTunnel: "https://bcb7-34-133-238-123.ngrok-free.app" -> "http://localhost:8050"
